![title](feature_types.png)

In [ ]:
import pandas as pd

data_orig =  pd.read_csv("trinket_data.csv", index_col=0)
y_orig = data_orig["price"]
X_orig = data_orig.drop("price", axis=1)

X_orig.head()

In [ ]:
from pyreal.transformers import Transformer, OneHotEncoder, Mappings, MappingsOneHotDecoder, FeatureSelectTransformer

class ColorTransformer(Transformer):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, x):
        return self

    def transform(self, x):
        for col in self.columns:
            x[col] = x[col].apply(hex_to_color_name)
        return x

    def transform_explanation(self, explanation):
        return explanation

class MaxAggregator(Transformer):
    def __init__(self, columns):
        self.columns = columns

    def transform(self, x):
        column_name = "MAX("
        column_name += ",".join(self.columns)
        column_name += ")"
        x[column_name] = x[self.columns].max(axis=1)
        x = x.drop(self.columns, axis=1)
        return x

def hex_to_color_name(h):
    h = h.lstrip('#')
    rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
    return ["red", "green", "blue"][rgb.index(max(rgb))]

colorTransformer = ColorTransformer(columns = ["color"])
typeEncoder = OneHotEncoder(columns = ["type"])
colorEncoder = OneHotEncoder(columns = ["color"])
maxAggregator = MaxAggregator(columns=["weight", "height"])
mappings = Mappings.generate_mappings(categorical_to_one_hot={"type": {"type_foo": "foo", "type_bar": "bar", "type_foobar": "foobar"}})
typeDecoder = MappingsOneHotDecoder(mappings)
featureSelect = FeatureSelectTransformer(['age', 'type_bar', 'type_foo', 'type_foobar',
                                          'color_blue', 'color_green', 'color_red',
                                          'MAX(weight,height)'])

In [ ]:
from pyreal.transformers import fit_transformers, run_transformers
from sklearn.linear_model import LinearRegression

e_transformers = []#typeDecoder]
m_transformers = [colorTransformer, colorEncoder, maxAggregator, typeEncoder, featureSelect]
i_transformers = [colorTransformer]

fit_transformers(e_transformers + m_transformers, X_orig)
fit_transformers(i_transformers, X_orig)

X_model = run_transformers(e_transformers + m_transformers, X_orig)
model = LinearRegression()
model.fit(X_model[0:400], y_orig[0:400])
print(model.score(X_model[401:], y_orig[401:]))

In [ ]:
from pyreal.explainers.lfc import simple_counterfactual_contribution as scc
from pyreal.utils import visualize

explainer = scc.SimpleCounterfactualContribution(model, X_orig,
                                                 m_transformers=m_transformers,
                                                 e_transformers=e_transformers,
                                                 i_transformers=i_transformers,
                                                 fit_on_init=True)

explanation = explainer.produce(X_orig.iloc[0:1])
print(explanation)
print(X_orig.iloc[0])
X_interpret = explainer.convert_data_to_interpretable(X_orig.iloc[0:1])
print(X_interpret)
visualize.plot_top_contributors(explanation, select_by="absolute", values=X_interpret.iloc[0])